# Testscript.ipynb

A Jupyter notebook for testing the `ArchetypeBuildingWeather.py` module.
Doubles as a *"tutorial"* of sorts.

In [ ]:
## Import module

import ArchetypeBuildingWeather as abw

In [ ]:
## Initialize input parameters

shapefile_path = "data/NUTS-Mopo.shp"
raster_path = "data/gfa_res_curr_density_epsg4326.tif"
weather_start = "2008-06"
weather_end = "2008-06"

In [ ]:
## Try creating a `Shapefile` and inspect the uniform raster

import matplotlib.pyplot as plt
ec = (0,0,0,1) # Edge color for the vector plots (black)
fc = (0,0,0,0) # Face color for the vector plots (transparent)
lw = 0.2 # Line width for the vector plots

shp = abw.Shapefile(shapefile_path)

f = abw.plot_layout(shp, shp.raster, title="Shapefile")

In [ ]:
## Try creating the cutout

cutout = abw.prepare_cutout(shp, weather_start, weather_end)

In [ ]:
## Create weights for testing.

#import random

#weights = {'KU005':0.24, 'KU010':0.73, 'KU009':0.03,}
#weights = {lid:random.uniform(0,1) for lid in shp.data.index}
#weights = {lid:1 for lid in shp.data.index}
weights = {"FI": 1.0}

In [ ]:
## Try preparing the `layout` raster and inspect nonzero data.

raster, layout = abw.prepare_layout(shp, cutout, weights, raster_path, resampling=5)

In [ ]:
## Plot the raster and layout

#fig1 = abw.plot_layout(shp, raster, title="Raster", dpi=90) # CAUTION! THIS TAKES A WHILE TO PLOT!
fig2 = abw.plot_layout(shp, layout, title="Layout", dpi=90)

In [ ]:
## Process weather data for the cutout.

ambient_temperature, diffuse_irradiation, direct_irradiation = abw.process_weather(
    cutout, layout
)

In [ ]:
# Plot temperature

ambient_temperature.plot()

In [ ]:
# Plot diffuse irradiation

diffuse_irradiation.plot()

In [ ]:
# Plot direct irradiation

import pandas as pd

pd.DataFrame(direct_irradiation).plot()

## Test cutout time ranges

`ArchetypeBuildingWeather.py` `v1.1.0` supports reusing cutouts with the same root shapefile, as long as the desired timespan is contained in the cutout already.
This functionality is tested below, along with the `aggregate_weather` convenience function.

In [ ]:
## The following should process and subsequently plot only the three first days worth of data.

weather_start = "2008-08-01"
weather_end = "2008-08-02"
temp, dif, dir = abw.aggregate_weather(
    shapefile_path, weights, weather_start, weather_end, raster_path
)
pd.DataFrame(temp).plot()
pd.DataFrame(dif).plot()
pd.DataFrame(dir).plot()

In [ ]:
temp_C = temp - 273.15

temp_C.plot(
    figsize = (5,2),
    ylabel = "ambient temperature [C]",
    title = "Weighted average ambient temperature",
)
plt.savefig("figs/temp.svg", dpi=600)

In [ ]:
irr = dict()
for key, val in dir.items():
    irr["direct: " + key] = val
irr["diffuse"] = dif

pd.DataFrame(irr).plot(
    figsize = (5,2),
    legend = True,
    ylabel = "solar irradiation [W/m2]",
    title = "Weighted average solar irradiation"
)
plt.savefig("figs/irr.svg", dpi=600)

In [ ]:
## The following should fail due to weather_end exceeding cutout.

weather_start = "2008-12-30"
weather_end_too_far = "2009-01-01"
temp, dif, dir = abw.aggregate_weather(
    shapefile_path, weights, weather_start, weather_end_too_far, raster_path
)